In [25]:
import sys
sys.path.append("..")

In [26]:
map_name = "Duckietown-small_loop-v0"

In [27]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [28]:
from stable_baselines.common.vec_env import VecVideoRecorder, DummyVecEnv

from utils.wrappers import ObsWrapper, CropResizeWrapper, MyRewardWrapper
from utils.duckie_wrappers import NormalizeWrapper, ImgWrapper, DtRewardWrapper, ActionWrapper, ResizeWrapper

import gym
import gym_duckietown


def record_video(model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: MyRewardWrapper(ObsWrapper(CropResizeWrapper(gym.make(map_name))))])
  # Start the video at step=0 and record 500 steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, reward, done, _ = eval_env.step(action)
    if done and reward < 0:
      print("*** CRASHED ***")
      print(reward)
    elif done:
      print("SAFE")
      print(reward)
    

  # Close the video recorder
  eval_env.close()

In [29]:
from pyvirtualdisplay import Display
from stable_baselines import DDPG
from stable_baselines import A2C

display = Display(visible=0, size=(1400, 900))
display.start()

model_to_be_loaded = "a2cDuckietown-small_loop-v060000.0"
model = A2C.load("../models/"+model_to_be_loaded)
record_video(model, video_length=500, prefix=map_name)

ipythondisplay.clear_output(wait=True)

/home/giulio/prog/.virtualenv/RL/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


*** CRASHED ***
[-50.]
Saving video to  /home/giulio/prog/duckiebot/notebooks/videos/Duckietown-small_loop-v0-step-0-to-step-500.mp4


In [30]:
show_videos('videos', prefix=map_name)